In [16]:
import pandas as pd
from auto_detect import DataAutoDetecter
from masking_funcs import Masker
from data_anonymizer import DataAnonymizer

In [17]:
df = pd.read_csv('adult.csv')

In [18]:
fakenric = pd.Series(range(1000000, 1048843)).map(lambda x : "T" + str(x) + "Z")

df["NRIC"] = fakenric

In [19]:
anonymizer = DataAnonymizer(df)

Auto Detecting all Data Properties...
Constructing Properties Frame
Auto-Generated Properties Frame. To change a property, use the change_property method.


In [20]:
anonymizer.properties_frame

,Column Type,Information Type,Sensitivity Type
Column Name,,,
age,Continuous,Others,Non-Sensitive
workclass,Categorical,Others,Non-Sensitive
fnlwgt,Continuous,Others,Non-Sensitive
education,Categorical,Others,Non-Sensitive
educational-num,Continuous,Others,Non-Sensitive
marital-status,Categorical,Others,Non-Sensitive
occupation,Categorical,Others,Non-Sensitive
relationship,Categorical,Others,Non-Sensitive
race,Categorical,Others,Non-Sensitive


In [21]:
anonymizer.properties_frame

,Column Type,Information Type,Sensitivity Type
Column Name,,,
age,Continuous,Others,Non-Sensitive
workclass,Categorical,Others,Non-Sensitive
fnlwgt,Continuous,Others,Non-Sensitive
education,Categorical,Others,Non-Sensitive
educational-num,Continuous,Others,Non-Sensitive
marital-status,Categorical,Others,Non-Sensitive
occupation,Categorical,Others,Non-Sensitive
relationship,Categorical,Others,Non-Sensitive
race,Categorical,Others,Non-Sensitive


In [22]:
anonymizer.change_property('educational-num', 'sensitivity_type', 'Sensitive')

,Column Type,Information Type,Sensitivity Type
Column Name,,,
age,Continuous,Others,Non-Sensitive
workclass,Categorical,Others,Non-Sensitive
fnlwgt,Continuous,Others,Non-Sensitive
education,Categorical,Others,Non-Sensitive
educational-num,Continuous,Others,Sensitive
marital-status,Categorical,Others,Non-Sensitive
occupation,Categorical,Others,Non-Sensitive
relationship,Categorical,Others,Non-Sensitive
race,Categorical,Others,Non-Sensitive


In [23]:
anonymizer.change_property('age', 
                        'sensitivity_type',
                        'Indirect Identifier')

anonymizer.change_property('gender', 
                        'sensitivity_type',
                        'Indirect Identifier')

anonymizer.change_property('native-country', 
                        'sensitivity_type',
                        'Indirect Identifier')

anonymizer.change_property('workclass', 
                        'sensitivity_type',
                        'Indirect Identifier')

anonymizer.change_property('education', 
                        'sensitivity_type',
                        'Indirect Identifier')

anonymizer.change_property('education', 
                        'column_type',
                        'Categorical')

anonymizer.change_property('capital-gain', 
                        'sensitivity_type',
                        'Indirect Identifier')

anonymizer.change_property('income', 
                        'sensitivity_type',
                        'Indirect Identifier')

anonymizer.change_property('capital-loss', 
                        'sensitivity_type',
                        'Indirect Identifier')

anonymizer.change_property('NRIC', 
                        'information_type',
                        'NRIC')

,Column Type,Information Type,Sensitivity Type
Column Name,,,
age,Continuous,Others,Indirect Identifier
workclass,Categorical,Others,Indirect Identifier
fnlwgt,Continuous,Others,Non-Sensitive
education,Categorical,Others,Indirect Identifier
educational-num,Continuous,Others,Sensitive
marital-status,Categorical,Others,Non-Sensitive
occupation,Categorical,Others,Non-Sensitive
relationship,Categorical,Others,Non-Sensitive
race,Categorical,Others,Non-Sensitive


In [24]:
anonymizer.get_mask_table() # autosuggest type of masking for each column (in order of priority) 
# a dictionary returned key = col_name,     val = recommended masking function

,Transformer
Column Name,
age,Generalise (Numerical Bin)
workclass,Encode
fnlwgt,Retain
education,Encode
educational-num,Generalise (Numerical Bin)
marital-status,Retain
occupation,Retain
relationship,Retain
race,Retain


In [25]:
anonymizer.list_allowed_transformations('fnlwgt')

['Retain',
 'Generalise (Numerical Bin)',
 'Generalise (Numerical Bin Mean)',
 'Surpress',
 'Pseudonymise',
 'Full Masking',
 'Transpose',
 'Shuffle']

In [26]:
anonymizer.change_masking(colname = 'age', masking_name = "Generalise (Numerical Bin Mean)") #masking algo
anonymizer.change_masking(colname = 'marital-status', masking_name = "Full Masking")
anonymizer.change_masking(colname = 'fnlwgt', masking_name = "Pseudonymise")
anonymizer.change_masking(colname = 'race', masking_name = "Shuffle")
anonymizer.change_masking(colname = 'hours-per-week', masking_name = "Generalise (Numerical Bin Mean)")
anonymizer.change_masking(colname = 'NRIC', masking_name = "Full Masking")

,Transformer
Column Name,
age,Generalise (Numerical Bin Mean)
workclass,Encode
fnlwgt,Pseudonymise
education,Encode
educational-num,Generalise (Numerical Bin)
marital-status,Full Masking
occupation,Retain
relationship,Retain
race,Shuffle


In [27]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,NRIC
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,T1000000Z
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,T1000001Z
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,T1000002Z
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,T1000003Z
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K,T1000004Z


In [28]:
anonymizer.apply_masking()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,NRIC
0,28.0,4,dc6104e6b688df78ba4abf3060d43758005e516e0fef2e...,1,"(6.0, 7.0]",-------------,Machine-op-inspct,Own-child,White,1,"(-100.0, 10000.0]","(-4.0, 436.0]",35.0,39,0,*****000Z
1,36.0,4,a4bce49b545d6ea7b117a3897c12d372ce9d3c7b64f34f...,11,"(8.0, 10.0]",------------------,Farming-fishing,Husband,White,1,"(-100.0, 10000.0]","(-4.0, 436.0]",45.0,39,0,*****001Z
2,28.0,2,8bf17dc8d786c16888079e8d241e17154a61ed087b4002...,7,"(12.0, 13.0]",------------------,Protective-serv,Husband,White,1,"(-100.0, 10000.0]","(-4.0, 436.0]",35.0,39,1,*****002Z
3,42.0,4,1a561f87377d353e18f915aaddf0fcfcc742b61f035355...,15,"(8.0, 10.0]",------------------,Machine-op-inspct,Husband,White,1,"(-100.0, 10000.0]","(-4.0, 436.0]",35.0,39,1,*****003Z
4,20.0,0,45abd026339c172e3feafc94980cf76b4052b03ad4af69...,15,"(8.0, 10.0]",-------------,?,Own-child,White,0,"(-100.0, 10000.0]","(-4.0, 436.0]",26.0,39,0,*****004Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,28.0,4,6cbd92497439d33e07977a21a6c4fec6e8ccf8773a7bd7...,7,"(12.0, 13.0]",------------------,Tech-support,Wife,Black,0,"(-100.0, 10000.0]","(-4.0, 436.0]",35.0,39,0,*****837Z
48838,42.0,4,50df42d528789d626a7f92f2da026bcf291134abd0348f...,11,"(8.0, 10.0]",------------------,Machine-op-inspct,Husband,White,1,"(-100.0, 10000.0]","(-4.0, 436.0]",35.0,39,1,*****838Z
48839,58.0,4,3439c79cef1cd8b692a298022db015dc5ae4334616259c...,11,"(8.0, 10.0]",-------,Adm-clerical,Unmarried,White,0,"(-100.0, 10000.0]","(-4.0, 436.0]",35.0,39,0,*****839Z
48840,20.0,4,a5ca8ce11e945b3866bf9a6dc8a8a19d050b65a8d7b779...,11,"(8.0, 10.0]",-------------,Adm-clerical,Own-child,White,1,"(-100.0, 10000.0]","(-4.0, 436.0]",16.0,39,0,*****840Z
